# Allora Forge Builder Kit

This notebook with help you get started submitting inferences to the Allora network and take part in the Forge competition.

## In this notebook you will quickly be able to:

  1. Load training and validation data.
  2. Load a LightGBM model
  3. Train the model
  4. Evaluate the model on validation data (recent 6 months)
  5. Package your predict function into a `.pkl` file
  6. Run your worker node to deliver inferences to Allora's network in real time

## What you will need

- **Allora Data API Key** (get your API key here: [https://developer.allora.network/](https://developer.allora.network/))

## Important: Allora Wallets

The last cell of this notebook runs a **worker** node, whose job it is to provide inferences to the Allora network. It is part of [Allora's Python SDK](https://github.com/allora-network/allora-sdk-py) and installs with `pip install allora-sdk`.

Code Example
```
from allora_sdk.worker import AlloraWorker

worker = AlloraWorker(
    predict_fn=predict_fn,
    api_key=api_key,
)
```

The worker automatically handles wallets. If you don't have a wallet, just hist <ENTER> when prompted for your menmonic phrase, and **a new wallet will be created automatically** and stored in the file marked `.allora_key`. If the key file exists, the worker will just run using that file. If you wish to enter a new mnemonic pass phrase, rename the `.allora_key` file to something else, the system will promopt you again for a new menmonic pass phrase.

Your wallet's inportant attributes are

 1. The Address
 2. The Menmonic Phrase (secret)

The address is public. It identifies your worker on Allora's network. Browse the network with Allora's [network explorer](https://explorer.allora.network/) to find your wallet address. Never tell anyone your mnemonic pass phrase, only use it to run your worker and interact with the blockchain.

**Important**: Keep track of your wallet details. The address is what you will use to participate in the Forge competitions and get access to your funds later.

# Happy Coding

Let's go.

## Install all necessary packages

In [ ]:
'''
Install all necessary packages
'''

%pip install git+https://github.com/allora-network/allora-forge-builder-kit.git
%pip install allora_sdk>=1.0.5 lightgbm scikit-learn pandas numpy matplotlib dill cloudpickle

# Optional display-only sanity echo
import os
if os.getenv("ALLORA_WALLET_ADDR"):
    print(f"Sanity (display only): ALLORA_WALLET_ADDR={os.getenv('ALLORA_WALLET_ADDR')}")

## Imports

The following libraries are required for this workflow:
- `allora_forge_builder_kit` for interfacing with the Allora Forge ML Workflow.
- `allora_sdk` for communicating with the Allora network
- `lightgbm` for model training.
- `pandas` and `numpy` for data manipulation.
- `cloudpickle` for serialization.
- `matplotlib` for visualization.
- `time` for timing operations.

These will be imported in the next code cell.

In [ ]:
from allora_forge_builder_kit import AlloraMLWorkflow, get_api_key
import lightgbm as lgb
import pandas as pd
import cloudpickle
import time
import matplotlib.pyplot as plt
import numpy as np

## API KEY
This gives you access to OHLCV (open, high, low, close, volume) candle data through the workflow.

To get your API key, go to [https://developer.allora.network/](https://developer.allora.network/), create an account, and generate a new API key.

In [ ]:
api_key = get_api_key()

## Initializing the Workflow
### Defining Assets, Input Feature Window, Target

Before training a model, we need to specify how much historical data to use as input features, and how far into the future we want to predict (the target). In this section, we define the parameters that control the *shape* and *amount* of our input features, which are simply normalized historical OHLCV candles.

- **tickers**: The set of tickers for data.
- **hours_needed**: This sets the total length (in hours) of the lookback window, i.e., how much historical context the model will see for each prediction.
- **number_of_input_candles**: This determines how many candles (or chunks) the lookback window is divided into. Each candle summarizes a portion of the lookback window, and together they form the input features.
- **target_length**: This specifies how many hours into the future we want to predict. For the Allora Forge competition, this should match the target horizon required by the challenge.

For example:

- Setting `hours_needed = 72` and `number_of_input_candles = 3` gives **3 daily candles** as input (each candle covers 24 hours).
- Setting `hours_needed = 24` and `number_of_input_candles = 24` gives **24 hourly candles** as input (each candle covers 1 hour).

We'll visualize the resulting features a few cells below, so you can see exactly what the model will receive as input.

In [ ]:
ticker = "btcusd" #eg btcusd, ethusd, solusd
tickers = [ticker] # for multiple tickers: ['btcusd', 'ethusd', 'solusd']
hours_needed = 5*24             # Number of historical hours for feature lookback window
number_of_input_candles = 12    # Number of candles for input features
target_length = 24            # Number of hours into the future for target (depends on forge competition)

# Instantiate the workflow
workflow = AlloraMLWorkflow(
    data_api_key=api_key,
    tickers=tickers,
    hours_needed=hours_needed,
    number_of_input_candles=number_of_input_candles,
    target_length=target_length
)

## Getting Data
### Training, Validation, and Test Data for Supervised Learning

The `get_train_validation_test_data` function splits your historical data into three consecutive time periods: **training**, **validation**, and **test**. This is a standard approach in supervised machine learning to ensure that your model is evaluated on data it has never seen during training.

### Arguments:
- **`from_month`**: The starting month (YYYY-MM) for the entire dataset. The earliest available is `"2020-01"`.
- **`validation_months`**: Number of months to reserve for the validation set (used for tuning hyperparameters and early stopping).
- **`test_months`**: Number of months to reserve for the test set (used for final evaluation).

### Data Splits:
The data is split chronologically:
1. **Training Set**: Includes all data from `from_month` up to the start of the validation period.
2. **Validation Set**: Immediately follows the training set, with an embargo (gap) equal to the prediction target length (e.g., 24 hours) to prevent data leakage.
3. **Test Set**: Follows the validation set, again separated by an embargo of the same length.

### How Long Does it Take?
Loading data should only take a few minutes, and versions of data are cached for quicker load times in the future.
- Force a re-download of fresh data with `force_redownload=True`.

### Visual Schematic:
For a 24-hour target:

| **Training Set** | → **24 hr** → | **Validation Set** | → **24 hr** → | **Test Set** |


In [ ]:
# Get training, validation, and test data
X_train, y_train, X_val, y_val, X_test, y_test = workflow.get_train_validation_test_data(
    from_month="2023-01",
    validation_months=3,
    test_months=3,
    force_redownload=False  # Set to True to force re-download of fresh data
)

# Example: Check the shapes of the datasets
print("Training set:", X_train.shape, y_train.shape)
print("Validation set:", X_val.shape, y_val.shape)
print("Test set:", X_test.shape, y_test.shape)

## Visualizing Data Time Spans

The next cell visualizes the time spans covered by the training, validation, and test datasets. This helps to ensure that the data splits are correctly separated in time and that there is no data leakage between them.

In [ ]:
# Print start and end dates for each split
def print_split_dates(X, name):
    if isinstance(X.index, pd.MultiIndex):
        dates = X.index.get_level_values(0)
    else:
        dates = X.index
    print(f"{name} set: {dates.min()} to {dates.max()}")

print_split_dates(X_train, "Training")
print_split_dates(X_val, "Validation")
print_split_dates(X_test, "Test")

# Visualize the splits on a timeline
import matplotlib.dates as mdates

fig, ax = plt.subplots(figsize=(10, 2))

def plot_span(X, color, label):
    if isinstance(X.index, pd.MultiIndex):
        dates = X.index.get_level_values(0)
    else:
        dates = X.index
    ax.axvspan(dates.min(), dates.max(), color=color, alpha=0.4, label=label)

plot_span(X_train, 'tab:blue', 'Training')
plot_span(X_val, 'tab:orange', 'Validation')
plot_span(X_test, 'tab:green', 'Test')

ax.legend(loc='upper left')
ax.set_title("Train/Validation/Test Splits Timeline")
ax.set_xlabel("Date")
ax.xaxis.set_major_locator(mdates.AutoDateLocator())
ax.xaxis.set_major_formatter(mdates.ConciseDateFormatter(ax.xaxis.get_major_locator()))
plt.tight_layout()
plt.show()

## Visualizing Features and Historical Price Process

In the next cell, we select a single row of data and visualize the features associated with it. These features represent the normalized historical OHLCV (open, high, low, close, volume) candles leading up to the selected time index.

To provide additional context, we overlay the historical price process during the same time period. This allows us to visually compare the input features with the actual price movements of the asset, offering insights into the relationship between the features and the price process.


In [ ]:
this_data = X_train.loc[(slice(None), 'btcusd'), :]

data_idx = -10000  # which data point to visualize

fig, ax1 = plt.subplots(figsize=(10, 6))

# Plot features based on data_idx
ax1.plot([this_data.iloc[data_idx][f"feature_open_{i}"] for i in range(number_of_input_candles)], 'o', label="open")
ax1.plot([this_data.iloc[data_idx][f"feature_high_{i}"] for i in range(number_of_input_candles)], 'o', label="high")
ax1.plot([this_data.iloc[data_idx][f"feature_low_{i}"] for i in range(number_of_input_candles)], 'o', label="low")
ax1.plot([this_data.iloc[data_idx][f"feature_close_{i}"] for i in range(number_of_input_candles)], 'o', label="close")
ax1.set_xlabel("Candle Index")
ax1.set_ylabel("Normalized Price Features")
ax1.legend(loc="upper left")

# Plot past price process based on data_idx
plt.twiny()
plt.plot(
    (this_data.iloc[data_idx - (hours_needed * 12):data_idx]['close'].values) / this_data.iloc[data_idx]['close'],
    label="5 min close (past)"
)
plt.legend(loc='center left')

# Plot volume as a bar chart with independent y-axis
ax2 = ax1.twinx()
ax2.bar(range(number_of_input_candles),
    [this_data.iloc[data_idx][f"feature_volume_{i}"] for i in range(number_of_input_candles)],
    alpha=0.3, color='gray', label="volume")
ax2.set_ylabel("Normalized Volume")
ax2.legend(loc="upper right")

plt.title("Features and Volume")
plt.show()


## Visualizing the Target

In [ ]:
this_data = X_train.loc[(slice(None), 'btcusd'), :]

data_idx = 80000  # which data point to visualize

fig, ax = plt.subplots(figsize=(12, 6))

# Historical price process (normalized so last value is 1)
hist_prices = this_data.iloc[data_idx - (hours_needed * 12):data_idx]['close']
hist_times = this_data.iloc[data_idx - (hours_needed * 12):data_idx].index.get_level_values(0)
if not hist_prices.empty:
    hist_norm = hist_prices / hist_prices.iloc[-1]
else:
    hist_norm = hist_prices

# Future price process (normalized so first value is 1)
selected_time = this_data.iloc[data_idx].name[0] if isinstance(this_data.index, pd.MultiIndex) else this_data.index[data_idx]
future_time = selected_time + pd.Timedelta(hours=target_length)
future_prices = this_data['close']
future_mask = (future_prices.index.get_level_values(0) > selected_time) & \
              (future_prices.index.get_level_values(0) <= future_time)
future_price_series = future_prices[future_mask]
if not future_price_series.empty:
    future_norm = future_price_series / future_price_series.iloc[0]
else:
    future_norm = future_price_series

# Plot historical and future price process on the same axis
ax.plot(hist_times, hist_norm.values, label="5 min close (past)", color='blue')
ax.plot(future_price_series.index.get_level_values(0), future_norm.values, label="5 min close (future)", color='green')
ax.axhline(1, color='gray', linestyle='--', label='Transition Point')
ax.set_xlabel("Time")
ax.legend(loc='center left')

# Overlay target value
target_value = y_train.loc[(selected_time, 'btcusd')] if isinstance(y_train.index, pd.MultiIndex) else y_train[selected_time]
ax.scatter([future_time], [1 + target_value], color='red', label='Target Value', zorder=5)
ax.legend(loc='upper left')

plt.title("Historical & Future Price Process, and Target Value")
plt.show()


## "Hello World!" ML Model

- This notebook demonstrates a simple **LightGBM model** that predicts the future price of an asset based on its past price process, as represented by the features.
- The **validation set** is used to:
    - Tune the model.
    - Perform **early stopping**, which helps prevent overfitting.
- The **test set** is used to evaluate the model's performance.

### Workflow Summary:
1. The model is trained on the **training set**.
2. Early stopping is applied using the **validation set** to determine the best iteration.
3. The model is retrained on the combined **training and validation data** using the best iteration found during training.
4. Finally, the model is evaluated on the **test set**.

---

### Future Directions:
- This is just one example of a modeling workflow.
- This is where a lot of **modeling magic** can happen:
    - Experiment with swapping in new techniques and models.
    - Try different feature engineering strategies.
    - Explore alternative evaluation metrics.
- Stay tuned! More notebooks will be available in the future with **different ideas and approaches** to inspire your modeling journey.

In [ ]:
#define feature columns and ML model
feature_cols = [f for f in list(X_train) if 'feature' in f]

#define hyperparameters for the LightGBM model
learning_rate = 0.1
max_depth = 5
num_leaves = 16

# Initialize LightGBM model with hyperparameters
model = lgb.LGBMRegressor(
    n_estimators=2000,  # Set a high number of estimators for early stopping
    learning_rate=learning_rate,
    max_depth=max_depth,
    num_leaves=num_leaves
)

# Define custom evaluation metric for correlation
def corr_eval_metric(y_true, y_pred):
    corr = np.corrcoef(y_true, y_pred)[0, 1]
    # LightGBM expects: (eval_name, eval_result, is_higher_better)
    return 'corr', corr, True

# Fit model with early stopping
model.fit(
    X_train[feature_cols], y_train,
    # eval_set=[(X_val[feature_cols], y_val)],
    # eval_metric=corr_eval_metric,
    # callbacks=[
    #     lgb.early_stopping(stopping_rounds=100, verbose=True)
    # ]
)
# best_iteration = model.best_iteration_

# #retrain the model to best iteration on train + validation data
# model = lgb.LGBMRegressor(
#     n_estimators=best_iteration,  # Use the optimal number of trees
#     learning_rate=learning_rate,
#     max_depth=max_depth,
#     num_leaves=num_leaves
# )
model.fit(
    pd.concat([X_train[feature_cols], X_val[feature_cols]]),
    pd.concat([y_train, y_val])
)
# Evaluate
#  on the test data
test_preds = model.predict(X_test[feature_cols])
test_preds = pd.Series(test_preds, index=X_test.index)


## The Workflow has a built-in method to evaluate the test data predictions.

In [ ]:
#show test metrics
metrics = workflow.evaluate_test_data(test_preds)
print(metrics)

#check directional accuracy for only predictions over a certain threshold, like 0.01
threshold = .02
good_data_index = np.abs(test_preds) > threshold
good_preds = test_preds[good_data_index]
good_trues = y_test[good_data_index]
da = np.mean(np.sign(good_preds) == np.sign(good_trues))
print(da)

## Retraining for live deployment
The model is finally re-trained on the entire dataset, including the test data,
before it is packaged into a pickled function for live predictions.

The `workflow.get_live_features` function creates a version of the features based on the most up-to-date live data, in exactly the same way that the historical data features were created, ensuring the model is getting coherent live data.

In [ ]:
model.fit(
    pd.concat([X_train[feature_cols], X_val[feature_cols], X_test[feature_cols]]),
    pd.concat([y_train, y_val, y_test])
)

# Final predict function
def predict() -> pd.Series:
    live_features = workflow.get_live_features("btcusd")
    preds = model.predict(live_features)
    return float( np.array( preds ) [0] )

# Pickle the function
with open("predict.pkl", "wb") as f:
    cloudpickle.dump(predict, f)

## Test Live Predictions

This function simulates a live prediction scenario by loading the pickled function,
calling it, and printing the prediction time and result.

In [ ]:
# Load the pickled predict function
with open("predict.pkl", "rb") as f:
    predict_fn = cloudpickle.load(f)

# Call the function and get predictions
tic = time.time()
prediction = predict_fn()
toc = time.time()

print("predict time: ", (toc - tic) )
print("prediction: ", prediction )


## 🚀 Run Your Worker Node

The following code loads your pickled prediction function and submits predictions to the Allora network at the required intervals.

When you are asked for your mnemonic, you can simply hit &lt;ENTER&gt; and the worker will generate one automatically.

In [ ]:
import asyncio
from allora_sdk.worker import AlloraWorker

async def main():
    with open("predict.pkl", "rb") as f:
        predict_fn = cloudpickle.load(f)

    worker = AlloraWorker(
        predict_fn=predict_fn,
        api_key=api_key,
    )

    async for result in worker.run():
        if isinstance(result, Exception):
            print(f"error: {str(result)}")
        else:
            print("Prediction submitted to Allora.")

await main()